# Lab 08 - Deploying and Serving Models
In this lab we will experiment with deploying a model as a pipiline with Flask.

We’ll work with a Twitter dataset in this section. Our aim is to detect hate speech in Tweets. For the sake of simplicity, we say a Tweet contains hate speech if it has a racist or sexist sentiment associated with it. We will create a web page that will contain a text box like this (users will be able to search for any text).

This lab was adopted from: https://www.analyticsvidhya.com/blog/2020/04/how-to-deploy-machine-learning-model-flask/

Let’s start by importing some of the required libraries.

In [1]:
# importing required libraries
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

Next, we will read the dataset and view the top rows

In [6]:
data = pd.read_csv('dataset/twitter_sentiments.csv')

In [7]:
data.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [8]:
data.shape

(31962, 3)

In [9]:
data.label.value_counts()

0    29720
1     2242
Name: label, dtype: int64

Now, we will divide the data into train and test using the scikit-learn train_test_split function. We will take only 20 percent of the data for testing purposes. We will stratify the data on the label column so that the distribution of the target label will be the same in both train and test data:

In [15]:
 #stratify is set to data['label'], which means that the class labels in the data DataFrame will be used to ensure 
#that the training and testing datasets have the same proportion of each label. This is useful when dealing with imbalanced
#datasets, where one or more classes may be underrepresented in the data.

train, test = train_test_split(data, test_size = 0.2, stratify = data['label'], random_state=21)

In [16]:
train.shape, test.shape

((25569, 3), (6393, 3))

In [21]:
#outputs the label distribution in the train set.
#The normalize=True argument normalizes the counts to give the proportion of
#each unique value in the dataset
train.label.value_counts(normalize=True)

0    0.929837
1    0.070163
Name: label, dtype: float64

In [41]:
#outputs the label distribution in the test set.
test.label.value_counts(normalize=True)

0    0.929923
1    0.070077
Name: label, dtype: float64

Note:Term Frequency (TF): It measures the frequency of a word in a document. It is calculated by dividing the number of occurrences of a word in a document by the total number of words in the document.

TF = (Number of times the word appears in a document) / (Total number of words in the document)

Inverse Document Frequency (IDF): It measures the rarity of a word in the corpus of documents. It is calculated by taking the logarithm of the total number of documents in the corpus divided by the number of documents containing the word.

IDF = log_e(Total number of documents in the corpus / Number of documents containing the word)

The TF-IDF value of a word is the product of its TF and IDF values. It is a measure of how important a word is in a document with respect to the entire corpus of documents.

-----------------------------------------------------
Now, we will create a TF-IDF vector of the tweet column using the TfidfVectorizer and we will pass the parameter lowercase as True so that it will first convert text to lowercase. We will also keep max features as 1000 and pass the predefined list of stop words present in the scikit-learn library.

First, create the object of the TFidfVectorizer, build your model and fit the model with the training data tweets:

In [44]:
# ENGLISH_STOP_WORDS = list(ENGLISH_STOP_WORDS)
tfidf_vectorizer = TfidfVectorizer(lowercase= True, max_features=1000, stop_words=ENGLISH_STOP_WORDS)

In [45]:
tfidf_vectorizer.fit(train.tweet)

TfidfVectorizer(max_features=1000,
                stop_words=['sometimes', 'when', 'up', 'we', 'whereafter',
                            'her', 'wherever', 'now', 'do', 'thru', 'each',
                            'ourselves', 'therein', 'above', 'please', 'if',
                            'anything', 'thick', 'anyone', 'being', 'twenty',
                            'everywhere', 'she', 'mill', 'you', 'cannot', 'all',
                            'onto', 'former', 'behind', ...])

Use the model and transform the train and test data tweets:

In [50]:
train_idf = tfidf_vectorizer.transform(train.tweet)
test_idf  = tfidf_vectorizer.transform(test.tweet)
#print(train_idf)


Now, we will create an object of the Logistic Regression model.

Remember – our focus is not on building a very accurate classification model but instead to see how we can deploy this predictive model to get the results.

In [51]:
model_LR = LogisticRegression()

In [52]:
model_LR.fit(train_idf, train.label)

LogisticRegression()

In [53]:
predict_train = model_LR.predict(train_idf)

In [54]:
predict_test = model_LR.predict(test_idf)

In [55]:
# f1 score on train data
f1_score(y_true= train.label, y_pred= predict_train)

0.4888178913738019

In [56]:
f1_score(y_true= test.label, y_pred= predict_test)

0.45751633986928114

Let’s define the steps of the pipeline:

Step 1: Create a TF-IDF vector of the tweet text with 1000 features as defined above

Step 2: Use a logistic regression model to predict the target labels

When we use the fit() function with a pipeline object, both steps are executed. Post the model training process, we use the predict() function that uses the trained model to generate the predictions.

Read more about sci-kit learn pipelines in this comprehensive article: [Build your first Machine Learning pipeline using scikit-learn](https://www.analyticsvidhya.com/blog/2020/01/build-your-first-machine-learning-pipeline-using-scikit-learn/)!

In [57]:
pipeline = Pipeline(steps= [('tfidf', TfidfVectorizer(lowercase=True,
                                                      max_features=1000,
                                                      stop_words= ENGLISH_STOP_WORDS)),
                            ('model', LogisticRegression())])

In [59]:
pipeline.fit(train.tweet, train.label)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=1000,
                                 stop_words=['sometimes', 'when', 'up', 'we',
                                             'whereafter', 'her', 'wherever',
                                             'now', 'do', 'thru', 'each',
                                             'ourselves', 'therein', 'above',
                                             'please', 'if', 'anything',
                                             'thick', 'anyone', 'being',
                                             'twenty', 'everywhere', 'she',
                                             'mill', 'you', 'cannot', 'all',
                                             'onto', 'former', 'behind', ...])),
                ('model', LogisticRegression())])

In [60]:
pipeline.predict(train.tweet)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

Now, we will test the pipeline with a sample tweet:

In [61]:
text = ["Virat Kohli, AB de Villiers set to auction their 'Green Day' kits from 2016 IPL match to raise funds"]

In [62]:
pipeline.predict(text)

array([0], dtype=int64)

We have successfully built the machine learning pipeline and we will save this pipeline object using the dump function in the joblib library. You just need to pass the pipeline object and the file name:

In [65]:
from joblib import dump

In [68]:
# dump() is a function from the joblib library used to serialize and save Python objects to a file. 
# In the given code, the pipeline object is being serialized and saved to a file named text_classification.joblib.

dump(pipeline, filename="text_classification.joblib")

['text_classification.joblib']

It will create a file name “text_classification.joblib“. Now, we will open another Python file and use the load function of the joblib library to load the pipeline model.

Let’s see how to use the saved model:

In [69]:
import pandas as pd
from joblib import load

In [70]:
text = ["Virat Kohli, AB de Villiers set to auction their 'Green Day' kits from 2016 IPL match to raise funds"]

In [71]:
pipeline = load("text_classification.joblib")

In [72]:
pipeline.predict(text)

array([0], dtype=int64)

In [73]:
data[data.label == 1]

,id,label,tweet
13,14,1,@user #cnn calls #michigan middle school 'buil...
14,15,1,no comment! in #australia #opkillingbay #se...
17,18,1,retweet if you agree!
23,24,1,@user @user lumpy says i am a . prove it lumpy.
34,35,1,it's unbelievable that in the 21st century we'...
...,...,...,...
31934,31935,1,lady banned from kentucky mall. @user #jcpenn...
31946,31947,1,@user omfg i'm offended! i'm a mailbox and i'...
31947,31948,1,@user @user you don't have the balls to hashta...
31948,31949,1,"makes you ask yourself, who am i? then am i a..."


Its now time to run the pipeline (i.e. data featurisation and model prediction) and make calls from a web page!

The following command will start the flask app as a python command... but ideally you would run this from a command line, not from the notebook.

In [ ]:
!python get_sentiment.py

Now that this is running go to  http://127.0.0.1:5000 or http://localhost:5000 and try it out

To stop the process just interrupt the kernel.